In [1]:
# Create the Dataset from detailed_listings

In [2]:
import pandas as pd
import numpy as np
import json
import pandas as pd
import re
from sklearn.model_selection import train_test_split


In [3]:
# Load your dataset
df = pd.read_csv("detailed_listings_berlin.csv")

# Clean 'price' column by removing $ and , then convert to float
df["price"] = df["price"].replace(r'[\$,]', '', regex=True).astype(float)

# Remove rows where 'price' is NaN (if any remain after conversion)
df = df[df["price"].notna()]

# Check resulting shape
print(df[["id","price"]])

                        id  price
0                     3176  105.0
1                     9991  135.0
2                    14325   75.0
3                    16644   77.0
4                    17904   40.0
...                    ...    ...
13940  1376348748790549852  107.0
13941  1376356410353693375  103.0
13942  1376569769868596901   59.0
13943  1376576995944779676  100.0
13944  1376671293707752704   71.0

[8898 rows x 2 columns]


### Help function

In [4]:
def get_amenities_at_index(df, index):
    raw_string = df.loc[index, "amenities"]
    amenities_list = json.loads(raw_string)
    return amenities_list

### Refridgerator

In [5]:
# First make price numeric

In [6]:
# Create new columns based on "amenities","description","name"

In [7]:
def create_refrigerator_columns(df):
    brand_column_names = {
        'lg': 'refrigerator_lg',
        'siemens': 'refrigerator_siemens',
        'gaggenau': 'refrigerator_gaggenau',
        'electrolux': 'refrigerator_electrolux',
        'liebherr': 'refrigerator_liebherr',
        'bosch': 'refrigerator_bosch',
        'miele': 'refrigerator_miele',
        'boomann': 'refrigerator_bomann',
        'sharp': 'refrigerator_sharp',
        'haier': 'refrigerator_haier',
        'whirlpool': 'refrigerator_whirlpool',
        'aeg': 'refrigerator_aeg',
        'ikea': 'refrigerator_ikea',
        'amazon': 'refrigerator_amazon',
        'samsung': 'refrigerator_samsung',
        'elektrolux': 'refrigerator_electrolux',
        'beko': 'refrigerator_beko',
        'beco': 'refrigerator_beko',
        'smeg': 'refrigerator_smeg',
        'severin': 'refrigerator_severin',
        'teka': 'refrigerator_teka',
        'zanussi': 'refrigerator_zanussi',
        'panasonic': 'refrigerator_panasonic',
        'vestel': 'refrigerator_vestel',
        'gorenje': 'refrigerator_gorenje',
        'diemens': 'refrigerator_siemens',
        'bomann': 'refrigerator_bomann',
        'amica': 'refrigerator_amica',
        'neff': 'refrigerator_neff',
    }

    new_cols = {
        "has_refrigerator": pd.Series(False, index=df.index),
        "refrigerator_shared": pd.Series(False, index=df.index),
    }
    
    for col in set(brand_column_names.values()):
        new_cols[col] = pd.Series(False, index=df.index)

    brand_keys_lower = {k.lower(): v for k, v in brand_column_names.items()}
    fridge_terms = ["refrigerator", "refridgerator", "fridge", "kühlschrank","kuehlschrank"]

    shared_fridge_pattern = re.compile(
        r"\b(shared fridge|shared refrigerator|community fridge|community refridgerator|geteilter kühlschrank|geteilter kuehlschrank)"
    )

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        # ✅ Detect has_refrigerator
        if any(term in a for term in fridge_terms for a in amenities_lower):
            new_cols["has_refrigerator"].at[idx] = True

        # ✅ Detect refrigerator_shared
        if any(shared_fridge_pattern.search(a) for a in amenities_lower):
            new_cols["refrigerator_shared"].at[idx] = True

        # ✅ Detect brand-specific columns
        for amenity_lower in amenities_lower:
            if any(term in amenity_lower for term in fridge_terms):
                for brand_key_lower, col_name in brand_keys_lower.items():
                    if brand_key_lower in amenity_lower:
                        new_cols[col_name].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df

In [8]:
def count_refrigerator_brands(df):
    # Identify all refrigerator brand columns in the DataFrame
    brand_columns = [col for col in df.columns if col.startswith("refrigerator_") and col != "has_refrigerator"]

    # Count the number of True values per column
    brand_counts = df[brand_columns].sum().sort_values(ascending=False)

    # Return as a clean DataFrame for inspection or reporting
    return brand_counts.reset_index().rename(columns={"index": "brand_column", 0: "count"})

In [9]:
def count_refrigerators_with_and_without_brand(df):
    # Identify all refrigerator brand columns
    brand_columns = [col for col in df.columns if col.startswith("refrigerator_") and col != "has_refrigerator"]

    # Boolean series: at least one brand column is True
    has_brand = df[brand_columns].any(axis=1)

    # Count:
    # Listings with refrigerator and with brand
    with_brand = ((df["has_refrigerator"]) & (has_brand)).sum()

    # Listings with refrigerator and without brand
    without_brand = ((df["has_refrigerator"]) & (~has_brand)).sum()

    return {"with_brand": int(with_brand), "without_brand": int(without_brand)}

In [10]:
df = create_refrigerator_columns(df)
brand_counts_df = count_refrigerator_brands(df)
result = count_refrigerators_with_and_without_brand(df)
print(result)
print(brand_counts_df)

{'with_brand': 396, 'without_brand': 6433}
               brand_column  count
0      refrigerator_siemens     72
1        refrigerator_bosch     66
2        refrigerator_miele     35
3          refrigerator_aeg     33
4     refrigerator_liebherr     29
5      refrigerator_samsung     26
6         refrigerator_smeg     23
7    refrigerator_whirlpool     21
8         refrigerator_ikea     13
9         refrigerator_neff     11
10          refrigerator_lg     11
11       refrigerator_amica     10
12        refrigerator_beko      9
13  refrigerator_electrolux      8
14     refrigerator_gorenje      7
15     refrigerator_severin      4
16      refrigerator_vestel      3
17    refrigerator_gaggenau      3
18       refrigerator_haier      3
19       refrigerator_sharp      3
20      refrigerator_bomann      3
21     refrigerator_zanussi      2
22      refrigerator_amazon      1
23   refrigerator_panasonic      1
24        refrigerator_teka      1
25      refrigerator_shared      0


In [11]:
# Set index to check
test_index = 77

In [12]:
# Identify refrigerator brand columns (excluding 'has_refrigerator')
refrigerator_columns = [col for col in df.columns if col.startswith("refrigerator_") and col != "has_refrigerator"]

# Find all indices where any brand column is True
indices_with_brand = df[df[refrigerator_columns].any(axis=1)].index

# Take the first available index among those rows
index_to_check = indices_with_brand[test_index]  # or loop over more if you wish

# Filter to only the columns with True for this index
true_brand_columns = [col for col in refrigerator_columns if df.at[index_to_check, col] == True]

# Display amenities, has_refrigerator, and only the brand columns that are True
columns_to_display = ["amenities", "has_refrigerator"] + true_brand_columns

df_check = df.loc[[index_to_check], columns_to_display]

print(f"\n=== Index: {index_to_check} | Displaying amenities and detected refrigerator brand(s) ===")
print(df_check.to_string(max_colwidth=1500))


=== Index: 5375 | Displaying amenities and detected refrigerator brand(s) ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                amenities  has_refrigerator  refrigerator_gorenje
5375  ["Drying rack for clothing", "Shared backyard \u2013 Fully fenced", "Dishes and silverware", "Bose sound system with aux", "Bed linens", "Books and reading material", "Hangers", "TV", "Wifi", "Coffee", "Dishwasher", "Freezer", "Cooking basic

### Oven

In [13]:
def create_oven_columns(df):
    # ✅ Clean trade oven brands only (adapted list, expand as needed)
    brand_column_names = {
        'bosch': 'oven_bosch',
        'miele': 'oven_miele',
        'siemens': 'oven_siemens',
        'aeg': 'oven_aeg',
        'neff': 'oven_neff',
        'gaggenau': 'oven_gaggenau',
        'smeg': 'oven_smeg',
        'whirlpool': 'oven_whirlpool',
        'electrolux': 'oven_electrolux',
        'zanussi': 'oven_zanussi',
        'beko': 'oven_beko',
        'ikea': 'oven_ikea',
        'haier': 'oven_haier',
        'panasonic': 'oven_panasonic',
        'teka': 'oven_teka',
    }

    # Initialize columns
    new_cols = {
        "has_oven": pd.Series(False, index=df.index)
    }
    for col in set(brand_column_names.values()):
        new_cols[col] = pd.Series(False, index=df.index)

    # Lowercase brand mapping
    brand_keys_lower = {k.lower(): v for k, v in brand_column_names.items()}
    oven_terms = ["oven", "backofen", "forno"]

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        # Set has_oven if any oven term found
        if any(any(term in a for term in oven_terms) for a in amenities_lower):
            new_cols["has_oven"].at[idx] = True

        # Set brand columns if oven term + brand in the same amenity
        for amenity_lower in amenities_lower:
            if any(term in amenity_lower for term in oven_terms):
                for brand_key_lower, col_name in brand_keys_lower.items():
                    if brand_key_lower in amenity_lower:
                        new_cols[col_name].at[idx] = True

    # Add to df
    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df

In [14]:
def count_oven_brands(df):
    brand_columns = [col for col in df.columns if col.startswith("oven_") and col != "has_oven"]
    brand_counts = df[brand_columns].sum().sort_values(ascending=False)
    return brand_counts.reset_index().rename(columns={"index": "brand_column", 0: "count"})

def count_ovens_with_and_without_brand(df):
    brand_columns = [col for col in df.columns if col.startswith("oven_") and col != "has_oven"]
    has_brand = df[brand_columns].any(axis=1)
    with_brand = ((df["has_oven"]) & (has_brand)).sum()
    without_brand = ((df["has_oven"]) & (~has_brand)).sum()
    return {"with_brand": int(with_brand), "without_brand": int(without_brand)}

In [15]:
df = create_oven_columns(df)

# Get counts
oven_brand_counts_df = count_oven_brands(df)
oven_counts = count_ovens_with_and_without_brand(df)

print(oven_counts)
print(oven_brand_counts_df)

{'with_brand': 260, 'without_brand': 4531}
       brand_column  count
0        oven_bosch     60
1      oven_siemens     52
2        oven_miele     52
3          oven_aeg     31
4         oven_neff     20
5         oven_ikea     17
6     oven_gaggenau      8
7         oven_smeg      6
8    oven_whirlpool      4
9      oven_zanussi      4
10        oven_beko      3
11  oven_electrolux      2
12        oven_teka      1
13   oven_panasonic      0
14       oven_haier      0


In [16]:
# Identify oven brand columns
oven_columns = [col for col in df.columns if col.startswith("oven_") and col != "has_oven"]

# Get indices where any oven brand was detected
indices_with_oven_brand = df[df[oven_columns].any(axis=1)].index

# Select the test index for inspection (adjust as needed)
test_index = 9 # or another index for targeted inspection
index_to_check = indices_with_oven_brand[test_index]

# Filter columns with True for this index
true_oven_columns = [col for col in oven_columns if df.at[index_to_check, col] == True]

# Display for double-check
columns_to_display = ["amenities", "has_oven"] + true_oven_columns
df_check = df.loc[[index_to_check], columns_to_display]

print(f"\n=== Index: {index_to_check} | Displaying amenities and detected oven brand(s) ===")
print(df_check.to_string(max_colwidth=1500))


=== Index: 390 | Displaying amenities and detected oven brand(s) ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  amenities  has_oven  oven_siemens
390  ["Bathtub", "Free dryer \u2013 In unit", "Drying rack for clothing", "Dishes and silverware", "Bed linens", "Siemens electric stove", "Paid street parkin

### Stove

In [17]:
def create_stove_columns(df):
    # ✅ Stove types detection patterns
    stove_types_patterns = {
        "stove_electric": r"\belectric\b",
        "stove_induction": r"\binduction\b",
        "stove_gas": r"\bgas\b",
    }

    # ✅ Stove brands to detect
    brand_column_names = {
        'bosch': 'stove_bosch',
        'miele': 'stove_miele',
        'siemens': 'stove_siemens',
        'aeg': 'stove_aeg',
        'neff': 'stove_neff',
        'gaggenau': 'stove_gaggenau',
        'smeg': 'stove_smeg',
        'whirlpool': 'stove_whirlpool',
        'electrolux': 'stove_electrolux',
        'zanussi': 'stove_zanussi',
        'beko': 'stove_beko',
        'ikea': 'stove_ikea',
        'haier': 'stove_haier',
        'panasonic': 'stove_panasonic',
        'teka': 'stove_teka',
    }

    # Initialize columns
    new_cols = {
        "has_stove": pd.Series(False, index=df.index)
    }
    for col in set(stove_types_patterns.keys()).union(brand_column_names.values()):
        new_cols[col] = pd.Series(False, index=df.index)

    # Lowercase mapping for brands
    brand_keys_lower = {k.lower(): v for k, v in brand_column_names.items()}

    # Stove detection terms
    stove_terms_pattern = re.compile(
        r"\b(stove|cooktop|herd|range|stovetop|cooktop)\b"
    )

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        # ✅ Check if stove term is present
        if any(stove_terms_pattern.search(a) for a in amenities_lower):
            new_cols["has_stove"].at[idx] = True

            # ✅ Check for types if stove term present
            for amenity_lower in amenities_lower:
                if stove_terms_pattern.search(amenity_lower):
                    for col_name, type_pattern in stove_types_patterns.items():
                        if re.search(type_pattern, amenity_lower):
                            new_cols[col_name].at[idx] = True

                    # ✅ Check for brands if stove term present
                    for brand_key_lower, col_name in brand_keys_lower.items():
                        if brand_key_lower in amenity_lower:
                            new_cols[col_name].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [18]:
df = create_stove_columns(df)

def count_stove_features(df):
    stove_columns = [col for col in df.columns if col.startswith("stove_")]
    stove_counts = df[stove_columns].sum().sort_values(ascending=False)
    return stove_counts.reset_index().rename(columns={"index": "stove_feature", 0: "count"})

def count_stoves_with_and_without_features(df):
    stove_columns = [col for col in df.columns if col.startswith("stove_") and col != "has_stove"]
    has_feature = df[stove_columns].any(axis=1)
    with_feature = ((df["has_stove"]) & (has_feature)).sum()
    without_feature = ((df["has_stove"]) & (~has_feature)).sum()
    return {"with_feature": int(with_feature), "without_feature": int(without_feature)}

In [19]:
# Identify stove columns
stove_columns = [col for col in df.columns if col.startswith("stove_")]

# Get indices where any stove type was detected
indices_with_stove_type = df[df[stove_columns].any(axis=1)].index

# Select test index
test_index = 20
index_to_check = indices_with_stove_type[test_index]

# Filter columns with True for this index
true_stove_columns = [col for col in stove_columns if df.at[index_to_check, col] == True]

# Display for validation
columns_to_display = ["amenities", "has_stove"] + true_stove_columns
df_check = df.loc[[index_to_check], columns_to_display]

print(f"\n=== Index: {index_to_check} | Displaying amenities and detected stove type(s) ===")
print(df_check.to_string(max_colwidth=1500))


=== Index: 90 | Displaying amenities and detected stove type(s) ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

### Sound system

In [20]:
import json
import pandas as pd

def get_amenities_at_index(df, index):
    raw_string = df.loc[index, "amenities"]
    amenities_list = json.loads(raw_string)
    return amenities_list

def create_sound_system_columns(df):
    # ✅ Clean trade brands for sound systems (expandable)
    brand_column_names = {
        'bose': 'sound_bose',
        'sonos': 'sound_sonos',
        'jbl': 'sound_jbl',
        'sony': 'sound_sony',
        'marshall': 'sound_marshall',
        'bang & olufsen': 'sound_bang_olufsen',
        'b&o': 'sound_bang_olufsen',
        'yamaha': 'sound_yamaha',
        'samsung': 'sound_samsung',
        'lg': 'sound_lg',
        'philips': 'sound_philips',
        'panasonic': 'sound_panasonic',
        'teufel': 'sound_teufel',
        'denon': 'sound_denon',
        'pioneer': 'sound_pioneer',
        'onkyo': 'sound_onkyo',
    }

    # Initialize columns
    new_cols = {
        "has_sound_system": pd.Series(False, index=df.index)
    }
    for col in set(brand_column_names.values()):
        new_cols[col] = pd.Series(False, index=df.index)

    # Lowercase brand mapping
    brand_keys_lower = {k.lower(): v for k, v in brand_column_names.items()}
    sound_terms = ["sound system", "stereo", "speaker", "speakers", "lautsprecher", "soundanlage", "hi-fi", "hifi", "soundbar"]

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        # Set has_sound_system if any sound-related term found
        if any(any(term in a for term in sound_terms) for a in amenities_lower):
            new_cols["has_sound_system"].at[idx] = True

        # Set brand columns if sound term + brand in the same amenity
        for amenity_lower in amenities_lower:
            if any(term in amenity_lower for term in sound_terms):
                for brand_key_lower, col_name in brand_keys_lower.items():
                    if brand_key_lower in amenity_lower:
                        new_cols[col_name].at[idx] = True

    # Add to df
    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [21]:
def count_sound_system_brands(df):
    brand_columns = [col for col in df.columns if col.startswith("sound_") and col != "has_sound_system"]
    brand_counts = df[brand_columns].sum().sort_values(ascending=False)
    return brand_counts.reset_index().rename(columns={"index": "brand_column", 0: "count"})

def count_sound_systems_with_and_without_brand(df):
    brand_columns = [col for col in df.columns if col.startswith("sound_") and col != "has_sound_system"]
    has_brand = df[brand_columns].any(axis=1)
    with_brand = ((df["has_sound_system"]) & (has_brand)).sum()
    without_brand = ((df["has_sound_system"]) & (~has_brand)).sum()
    return {"with_brand": int(with_brand), "without_brand": int(without_brand)}


In [22]:
df = create_sound_system_columns(df)

# Get counts
sound_brand_counts_df = count_sound_system_brands(df)
sound_counts = count_sound_systems_with_and_without_brand(df)

print(sound_counts)
print(sound_brand_counts_df)

{'with_brand': 279, 'without_brand': 751}
          brand_column  count
0          sound_sonos    106
1           sound_bose     45
2         sound_teufel     20
3         sound_yamaha     20
4       sound_marshall     19
5   sound_bang_olufsen     18
6           sound_sony     15
7            sound_jbl     11
8             sound_lg      9
9        sound_pioneer      7
10       sound_samsung      7
11       sound_philips      5
12     sound_panasonic      4
13         sound_onkyo      3
14         sound_denon      3


In [23]:
# Identify sound system brand columns
sound_columns = [col for col in df.columns if col.startswith("sound_") and col != "has_sound_system"]

# Get indices where any sound system brand was detected
indices_with_sound_brand = df[df[sound_columns].any(axis=1)].index

# Select test index
test_index = 10
index_to_check = indices_with_sound_brand[test_index]

# Filter columns with True for this index
true_sound_columns = [col for col in sound_columns if df.at[index_to_check, col] == True]

# Display for validation
columns_to_display = ["amenities", "has_sound_system"] + true_sound_columns
df_check = df.loc[[index_to_check], columns_to_display]

print(f"\n=== Index: {index_to_check} | Displaying amenities and detected sound system brand(s) ===")
print(df_check.to_string(max_colwidth=1500))


=== Index: 270 | Displaying amenities and detected sound system brand(s) ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      amenities  has_sound_system  sound_onkyo
270  ["Dishes and silverware", "Bed linens", "Long term stays allowed", "Hangers", "AEG refrigerator", "Wifi", "Dishwasher", "Other electric stove", "Cleaning products", "Onkyo sound system", "Window guards", "Coffee maker: drip coffee maker", "Cooking basics", "Essentials", "Hair d

### Coffee

In [24]:
import json
import pandas as pd
import re

def create_coffee_maker_columns(df):
    # Expanded detection patterns with regex-friendly variants
    coffee_types_patterns = {
        "coffee_espresso": r"\bespresso\b",
        "coffee_french_plus": r"\bfrench press\b",
        "coffee_nespresso": r"\bnespresso\b",
        "pour_over_coffee": r"\bpour[- ]over( coffee)?\b",
    }

    new_cols = {
        "has_coffee_maker": pd.Series(False, index=df.index)
    }
    for col in coffee_types_patterns.keys():
        new_cols[col] = pd.Series(False, index=df.index)

    # Terms for has_coffee_maker detection (expanded)
    coffee_terms_pattern = re.compile(r"\b(?:coffee|kaffee|espresso|french press|nespresso|pour[- ]over(?: coffee)?)\b")

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        # ✅ Precise has_coffee_maker detection
        if any(coffee_terms_pattern.search(a) for a in amenities_lower):
            new_cols["has_coffee_maker"].at[idx] = True

        # ✅ Precise per-type detection
        for amenity_lower in amenities_lower:
            for col_name, pattern in coffee_types_patterns.items():
                if re.search(pattern, amenity_lower):
                    new_cols[col_name].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [25]:
def count_coffee_maker_types(df):
    coffee_columns = [col for col in df.columns if col.startswith("coffee_") or col == "pour_over_coffee"]
    coffee_counts = df[coffee_columns].sum().sort_values(ascending=False)
    return coffee_counts.reset_index().rename(columns={"index": "coffee_type", 0: "count"})

def count_coffee_makers_with_and_without_types(df):
    coffee_columns = [col for col in df.columns if col.startswith("coffee_") or col == "pour_over_coffee"]
    has_type = df[coffee_columns].any(axis=1)
    with_type = ((df["has_coffee_maker"]) & (has_type)).sum()
    without_type = ((df["has_coffee_maker"]) & (~has_type)).sum()
    return {"with_type": int(with_type), "without_type": int(without_type)}

In [26]:
df = create_coffee_maker_columns(df)

# Get counts
coffee_counts_df = count_coffee_maker_types(df)
coffee_summary = count_coffee_makers_with_and_without_types(df)

print(coffee_summary)
print(coffee_counts_df)

{'with_type': 1857, 'without_type': 3870}
          coffee_type  count
0    coffee_nespresso    915
1     coffee_espresso    505
2  coffee_french_plus    386
3    pour_over_coffee    369


In [27]:
# Identify coffee columns
coffee_columns = [col for col in df.columns if col.startswith("coffee_") or col == "pour_over_coffee"]

# Get indices where any coffee type was detected
indices_with_coffee_type = df[df[coffee_columns].any(axis=1)].index

# Select test index for inspection
test_index = 4
index_to_check = indices_with_coffee_type[test_index]

# Filter columns with True for this index
true_coffee_columns = [col for col in coffee_columns if df.at[index_to_check, col] == True]

# Display for validation
columns_to_display = ["amenities", "has_coffee_maker"] + true_coffee_columns
df_check = df.loc[[index_to_check], columns_to_display]

print(f"\n=== Index: {index_to_check} | Displaying amenities and detected coffee type(s) ===")
print(df_check.to_string(max_colwidth=1500))


=== Index: 42 | Displaying amenities and detected coffee type(s) ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

### Excercise Equipment

In [28]:
import json
import pandas as pd
import re

def create_exercise_equipment_columns(df):
    # ✅ Define patterns for equipment types
    equipment_types_patterns = {
        "exercise_equipment_free_weights": r"\b(dumbbells?|free weights?|kettlebells?)\b",
        "exercise_equipment_elliptical": r"\belliptical\b",
        "exercise_equipment_stationary_bike": r"\b(stationary bike|exercise bike|spinning bike)\b",
        "exercise_equipment_yoga_mat": r"\b(yoga mat|yogamat)\b",
        "exercise_equipment_workout_bench": r"\b(workout bench|weight bench|training bench)\b",
        "exercise_equipment_treadmill": r"\btreadmill\b",
    }

    # Initialize columns
    new_cols = {
        "has_exercise_equipment": pd.Series(False, index=df.index)
    }
    for col in equipment_types_patterns.keys():
        new_cols[col] = pd.Series(False, index=df.index)

    # General terms for 'has_exercise_equipment'
    exercise_terms_pattern = re.compile(
        r"\b(gym|fitness|exercise|workout|training|dumbbell|weights?|kettlebell|elliptical|bike|yoga|treadmill|rowing)\b"
    )

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        # ✅ Set has_exercise_equipment if any relevant term detected
        if any(exercise_terms_pattern.search(a) for a in amenities_lower):
            new_cols["has_exercise_equipment"].at[idx] = True

        # ✅ Set equipment type columns precisely
        for amenity_lower in amenities_lower:
            if exercise_terms_pattern.search(amenity_lower):
                for col_name, pattern in equipment_types_patterns.items():
                    if re.search(pattern, amenity_lower):
                        new_cols[col_name].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [29]:
def count_exercise_equipment_types(df):
    equipment_columns = [col for col in df.columns if col.startswith("exercise_equipment_")]
    equipment_counts = df[equipment_columns].sum().sort_values(ascending=False)
    return equipment_counts.reset_index().rename(columns={"index": "equipment_type", 0: "count"})

def count_exercise_equipment_with_and_without_types(df):
    equipment_columns = [col for col in df.columns if col.startswith("exercise_equipment_") and col != "has_exercise_equipment"]
    has_type = df[equipment_columns].any(axis=1)
    with_type = ((df["has_exercise_equipment"]) & (has_type)).sum()
    without_type = ((df["has_exercise_equipment"]) & (~has_type)).sum()
    return {"with_type": int(with_type), "without_type": int(without_type)}


In [30]:
df = create_exercise_equipment_columns(df)

# Get counts
equipment_counts_df = count_exercise_equipment_types(df)
equipment_summary = count_exercise_equipment_with_and_without_types(df)

print(equipment_summary)
print(equipment_counts_df)

{'with_type': 220, 'without_type': 403}
                       equipment_type  count
0         exercise_equipment_yoga_mat    147
1     exercise_equipment_free_weights    126
2       exercise_equipment_elliptical     26
3  exercise_equipment_stationary_bike     21
4    exercise_equipment_workout_bench     17
5        exercise_equipment_treadmill     17


In [31]:
# Identify exercise equipment columns
equipment_columns = [col for col in df.columns if col.startswith("exercise_equipment_")]

# Get indices where any equipment detected
indices_with_equipment = df[df[equipment_columns].any(axis=1)].index

# Select test index
test_index = 10
index_to_check = indices_with_equipment[test_index]

# Filter columns with True for this row
true_equipment_columns = [col for col in equipment_columns if df.at[index_to_check, col] == True]

# Display for QA
columns_to_display = ["amenities", "has_exercise_equipment"] + true_equipment_columns
df_check = df.loc[[index_to_check], columns_to_display]

print(f"\n=== Index: {index_to_check} | Displaying amenities and detected exercise equipment features ===")
print(df_check.to_string(max_colwidth=2500))


=== Index: 485 | Displaying amenities and detected exercise equipment features ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

### Game Console

In [32]:
import json
import pandas as pd
import re

def create_game_console_columns(df):
    # ✅ Define regex patterns for each console
    console_patterns = {
        "ps2": r"\b(ps2|playstation 2)\b",
        "ps3": r"\b(ps3|playstation 3)\b",
        "ps4": r"\b(ps4|playstation 4)\b",
        "ps5": r"\b(ps5|playstation 5)\b",
        "xbox360": r"\b(xbox 360|xbox360)\b",
        "xboxone": r"\b(xbox one|xboxone)\b",
        "nintendoswitch": r"\b(nintendo switch|switch)\b",
        "wii": r"\b(wii|nintendo wii)\b",
    }

    # Initialize columns
    new_cols = {
        "has_game_console": pd.Series(False, index=df.index)
    }
    for col in console_patterns.keys():
        new_cols[col] = pd.Series(False, index=df.index)

    # General console detection pattern
    console_terms_pattern = re.compile(
        r"\b(playstation|ps[2-5]|xbox|nintendo|switch|wii|game console|gaming console)\b"
    )

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        # ✅ Set has_game_console if general term detected
        if any(console_terms_pattern.search(a) for a in amenities_lower):
            new_cols["has_game_console"].at[idx] = True

        # ✅ Set individual console columns if detected
        for amenity_lower in amenities_lower:
            if console_terms_pattern.search(amenity_lower):
                for col_name, pattern in console_patterns.items():
                    if re.search(pattern, amenity_lower):
                        new_cols[col_name].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [33]:
def count_game_console_types(df):
    console_columns = ["ps2", "ps3", "ps4", "ps5", "xbox360", "xboxone", "nintendoswitch", "wii"]
    console_counts = df[console_columns].sum().sort_values(ascending=False)
    return console_counts.reset_index().rename(columns={"index": "console_type", 0: "count"})

def count_game_consoles_with_and_without_type(df):
    console_columns = ["ps2", "ps3", "ps4", "ps5", "xbox360", "xboxone", "nintendoswitch", "wii"]
    has_type = df[console_columns].any(axis=1)
    with_type = ((df["has_game_console"]) & (has_type)).sum()
    without_type = ((df["has_game_console"]) & (~has_type)).sum()
    return {"with_type": int(with_type), "without_type": int(without_type)}

In [34]:
df = create_game_console_columns(df)

# Get counts
console_counts_df = count_game_console_types(df)
console_summary = count_game_consoles_with_and_without_type(df)

print(console_summary)
print(console_counts_df)

{'with_type': 88, 'without_type': 81}
     console_type  count
0             ps4     35
1  nintendoswitch     21
2             ps3     11
3             ps5     11
4             wii     11
5         xboxone      7
6         xbox360      3
7             ps2      2


In [35]:
# Identify console columns
console_columns = ["ps2", "ps3", "ps4", "ps5", "xbox360", "xboxone", "nintendoswitch", "wii"]

# Get indices where any console detected
indices_with_console = df[df[console_columns].any(axis=1)].index

# Select test index
test_index = 70
index_to_check = indices_with_console[test_index]

# Columns with True for this row
true_console_columns = [col for col in console_columns if df.at[index_to_check, col] == True]

# Display for QA
columns_to_display = ["amenities", "has_game_console"] + true_console_columns
df_check = df.loc[[index_to_check], columns_to_display]

print(f"\n=== Index: {index_to_check} | Displaying amenities and detected game console features ===")
print(df_check.to_string(max_colwidth=1500))


=== Index: 11587 | Displaying amenities and detected game console features ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

### TV

In [36]:
import json
import pandas as pd
import re


def create_tv_columns(df):
    # Initialize columns
    new_cols = {
        "has_tv": pd.Series(False, index=df.index),
        "tv_size_inch": pd.Series(None, index=df.index, dtype=float)
    }

    # TV detection pattern (detects tv, hdtv, smart tv, etc.)
    tv_terms_pattern = re.compile(
        r"\b(tv|television|smart tv|hdtv)\b"
    )

    # Pattern to extract TV size in inches
    tv_size_pattern = re.compile(
        r"(\d{2,3}(?:[.,]\d+)?)\s*(?:\"|inch|in|”)"
    )

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        found_tv = False
        sizes = []

        for amenity in amenities_lower:
            # Check if TV is mentioned
            if tv_terms_pattern.search(amenity):
                found_tv = True

                # Attempt to extract size
                size_match = tv_size_pattern.search(amenity)
                if size_match:
                    size_str = size_match.group(1).replace(",", ".")
                    try:
                        size_float = float(size_str)
                        sizes.append(size_float)
                    except ValueError:
                        pass

        if found_tv:
            new_cols["has_tv"].at[idx] = True

            # If multiple sizes are found, take the largest (most likely the primary TV)
            if sizes:
                new_cols["tv_size_inch"].at[idx] = max(sizes)

    # Add to df
    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [37]:
df = create_tv_columns(df)

In [38]:
def count_tv_summary(df):
    has_tv = df["has_tv"] == True
    has_size = df["tv_size_inch"].notna()

    with_size = (has_tv & has_size).sum()
    without_size = (has_tv & ~has_size).sum()
    total_tv = has_tv.sum()

    return {
        "total_tv": int(total_tv),
        "with_size": int(with_size),
        "without_size": int(without_size)
    }

def count_tv_size_distribution(df, bins=None):
    """
    Returns a distribution of TV sizes (using bins if provided).
    """
    tv_sizes = df.loc[df["tv_size_inch"].notna(), "tv_size_inch"]
    if bins:
        return pd.cut(tv_sizes, bins=bins).value_counts().sort_index()
    else:
        return tv_sizes.value_counts().sort_index()
        

In [39]:
# Get TV counts summary
tv_summary = count_tv_summary(df)

# Get TV size distribution (optional)
tv_size_distribution = count_tv_size_distribution(df)

# Display results
print(tv_summary)
print(tv_size_distribution)

{'total_tv': 6085, 'with_size': 825, 'without_size': 5260}
tv_size_inch
16.0       1
23.0       2
24.0       7
25.0       1
26.0       4
27.0       2
28.0       4
30.0      10
31.0       3
32.0     106
33.0       2
34.0       5
35.0      10
37.0       2
38.0       2
39.0       4
40.0      70
41.0       5
42.0      58
43.0     121
44.0       2
45.0      11
46.0      11
47.0       6
48.0      15
49.0      12
50.0      72
52.0       7
53.0       1
54.0       3
55.0     125
56.0       2
58.0       5
60.0      14
65.0      66
70.0       8
74.0       1
75.0      12
77.0       2
80.0       3
81.0       1
82.0       1
85.0       4
86.0       1
98.0       1
100.0      2
108.0      8
110.0      2
120.0      2
140.0      1
150.0      2
164.0      1
300.0      1
400.0      1
Name: count, dtype: int64


In [40]:
# Filter for rows with TV
indices_with_tv = df[df["tv_size_inch"].notna()].index

# Pick the nth row to inspect
test_index = 9
if test_index < len(indices_with_tv):
    index_to_check = indices_with_tv[test_index]
    df_check = df.loc[[index_to_check], ["amenities", "has_tv", "tv_size_inch"]]
    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected TV features ===")
    print(df_check.to_string(max_colwidth=1500))
else:
    print(f"Only {len(indices_with_tv)} rows with has_tv == True; test_index {test_index} is out of range.")


=== Index: 88 | Displaying amenities and detected TV features ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           amenities  has_tv  tv_size_inch
88  ["Dryer", "Dishes and silverware", "Bed linens", "Outdoor dining area", "Books and reading material", "Hangers", "Refrigerator", "Wifi", "Dishwasher", "Cleaning products", "Freezer", "Beach access \u2013 Beachfront", "Electric sto

### Streaming Provider

In [41]:
def create_streaming_provider_columns(df):
    # Streaming services detection patterns
    streaming_patterns = {
        "streaming_netflix": r"\b(netflix)\b",
        "streaming_amazon_prime_video": r"\b(amazon prime video|prime video|amazon prime)\b",
        "streaming_apple_tv": r"\b(apple tv|appletv)\b",
        "streaming_disney_plus": r"(disney\+|disney plus|disney\s*\+)"

    }

    # Initialize columns
    new_cols = {
        "has_streaming_provider": pd.Series(False, index=df.index)
    }
    for col in streaming_patterns.keys():
        new_cols[col] = pd.Series(False, index=df.index)

    # General detection pattern for has_streaming_provider
    streaming_terms_pattern = re.compile(
    r"(netflix|amazon prime video|prime video|amazon prime|apple tv|appletv|disney\+|disney plus|disney\s*\+)"
    )

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        # Flag has_streaming_provider if any relevant term detected
        if any(streaming_terms_pattern.search(a) for a in amenities_lower):
            new_cols["has_streaming_provider"].at[idx] = True

        # Flag specific streaming providers
        for amenity_lower in amenities_lower:
            if streaming_terms_pattern.search(amenity_lower):
                for col_name, pattern in streaming_patterns.items():
                    if re.search(pattern, amenity_lower):
                        new_cols[col_name].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [42]:
def count_streaming_provider_types(df):
    streaming_columns = [col for col in df.columns if col.startswith("streaming_")]
    streaming_counts = df[streaming_columns].sum().sort_values(ascending=False)
    return streaming_counts.reset_index().rename(columns={"index": "streaming_provider", 0: "count"})

def count_streaming_providers_with_and_without_types(df):
    streaming_columns = [col for col in df.columns if col.startswith("streaming_") and col != "has_streaming_provider"]
    has_type = df[streaming_columns].any(axis=1)
    with_type = ((df["has_streaming_provider"]) & (has_type)).sum()
    without_type = ((df["has_streaming_provider"]) & (~has_type)).sum()
    return {"with_type": int(with_type), "without_type": int(without_type)}

In [43]:
df = create_streaming_provider_columns(df)

# Get counts
streaming_counts_df = count_streaming_provider_types(df)
streaming_summary = count_streaming_providers_with_and_without_types(df)

print(streaming_summary)
print(streaming_counts_df)

{'with_type': 520, 'without_type': 0}
             streaming_provider  count
0             streaming_netflix    454
1  streaming_amazon_prime_video    252
2         streaming_disney_plus     97
3            streaming_apple_tv     93


In [44]:
# Identify streaming columns
streaming_columns = [col for col in df.columns if col.startswith("streaming_")]

# Get indices where any streaming service is detected
indices_with_streaming = df[df["has_streaming_provider"] == True].index

# Select test index
test_index = 3
if test_index < len(indices_with_streaming):
    index_to_check = indices_with_streaming[test_index]
    true_streaming_columns = [col for col in streaming_columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities", "has_streaming_provider"] + true_streaming_columns
    df_check = df.loc[[index_to_check], columns_to_display]
    
    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected streaming providers ===")
    print(df_check.to_string(max_colwidth=2000))
else:
    print(f"Only {len(indices_with_streaming)} rows with has_streaming_provider == True; test_index {test_index} is out of range.")


=== Index: 26 | Displaying amenities and detected streaming providers ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

### Wardrobe

In [45]:
import json
import pandas as pd
import re

def create_clothing_storage_columns(df):
    # Clothing storage patterns
    clothing_patterns = {
        "clothing_storage_closet": r"\bcloset\b",
        "clothing_storage_wardrobe": r"\bwardrobe\b",
        "clothing_storage_walk_in": r"\bwalk[- ]?in\b",
    }

    # Initialize columns
    new_cols = {
        "has_clothing_storage": pd.Series(False, index=df.index)
    }
    for col in clothing_patterns.keys():
        new_cols[col] = pd.Series(False, index=df.index)

    # General detection pattern for has_clothing_storage
    storage_terms_pattern = re.compile(
        r"\b(clothing storage|closet|wardrobe)"
    )

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        # Flag has_clothing_storage if any relevant term detected
        if any(storage_terms_pattern.search(a) for a in amenities_lower):
            new_cols["has_clothing_storage"].at[idx] = True

        # Flag specific types
        for amenity_lower in amenities_lower:
            if storage_terms_pattern.search(amenity_lower):
                for col_name, pattern in clothing_patterns.items():
                    if re.search(pattern, amenity_lower):
                        new_cols[col_name].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [46]:
def count_clothing_storage_types(df):
    clothing_columns = [col for col in df.columns if col.startswith("clothing_storage_")]
    clothing_counts = df[clothing_columns].sum().sort_values(ascending=False)
    return clothing_counts.reset_index().rename(columns={"index": "clothing_storage_type", 0: "count"})

def count_clothing_storage_with_and_without_types(df):
    clothing_columns = [col for col in df.columns if col.startswith("clothing_storage_") and col != "has_clothing_storage"]
    has_type = df[clothing_columns].any(axis=1)
    with_type = ((df["has_clothing_storage"]) & (has_type)).sum()
    without_type = ((df["has_clothing_storage"]) & (~has_type)).sum()
    return {"with_type": int(with_type), "without_type": int(without_type)}

In [47]:
df = create_clothing_storage_columns(df)

# Get counts
clothing_counts_df = count_clothing_storage_types(df)
clothing_summary = count_clothing_storage_with_and_without_types(df)

print(clothing_summary)
print(clothing_counts_df)

{'with_type': 2293, 'without_type': 1921}
       clothing_storage_type  count
0  clothing_storage_wardrobe   1672
1    clothing_storage_closet    836
2   clothing_storage_walk_in    117


In [48]:
# Identify clothing storage columns
clothing_columns = [col for col in df.columns if col.startswith("clothing_storage_")]

# Get indices where clothing storage detected
indices_with_clothing = df[df["has_clothing_storage"] == True].index

# Select test index
test_index = 4
if test_index < len(indices_with_clothing):
    index_to_check = indices_with_clothing[test_index]
    true_clothing_columns = [col for col in clothing_columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities", "has_clothing_storage"] + true_clothing_columns
    df_check = df.loc[[index_to_check], columns_to_display]
    
    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected clothing storage features ===")
    print(df_check.to_string(max_colwidth=2000))
else:
    print(f"Only {len(indices_with_clothing)} rows with has_clothing_storage == True; test_index {test_index} is out of range.")


=== Index: 23 | Displaying amenities and detected clothing storage features ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           amenities  has_clothing_storage  clothing_storage_wardrobe
23  ["Bathtub", "Free dryer \u2013 In unit", "Drying rack for clothing", "Dishes and silverware", "Bed linens", "Long term stays allowed", "Refrigerator", "Hangers", "Wifi", "High chair", "Cooking basics", "Essentials", "Coffee maker", "Hair dryer", "Extra pillows and blankets", "Smart lock", "Central heating", "Self check-in", "Kitchen", "Single leve

### Parking

In [49]:
import json
import pandas as pd
import re

def create_parking_columns(df):
    # Parking patterns
    parking_patterns = {
        "parking_free_carport": r"\bfree carport\b",
        "parking_free_driveway": r"\bfree driveway parking\b",
        "parking_free_garage": r"\bfree parking garage\b",
        "parking_free_residential_garage": r"\bfree residential garage\b",
        "parking_free_street": r"\bfree street parking\b",
        "parking_paid_garage": r"\bpaid parking garage\b",
        "parking_paid_lot": r"\bpaid parking lot\b",
        "parking_paid": r"\bpaid parking\b",
        "parking_paid_street": r"\bpaid street parking\b",
    }

    # Initialize columns
    new_cols = {
        "has_parking": pd.Series(False, index=df.index),
        "parking_on_premises": pd.Series(False, index=df.index),
        "parking_off_premises": pd.Series(False, index=df.index),
    }
    for col in parking_patterns.keys():
        new_cols[col] = pd.Series(False, index=df.index)

    # General parking detection
    parking_terms_pattern = re.compile(
        r"\b(parking|carport|driveway|garage)\b"
    )
    on_premises_pattern = re.compile(r"\bon[- ]premises\b")
    off_premises_pattern = re.compile(r"\boff[- ]premises\b")

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        # Check if any parking mention exists
        if any(parking_terms_pattern.search(a) for a in amenities_lower):
            new_cols["has_parking"].at[idx] = True

        # Check for on-premises / off-premises
        for amenity_lower in amenities_lower:
            if on_premises_pattern.search(amenity_lower):
                new_cols["parking_on_premises"].at[idx] = True
            if off_premises_pattern.search(amenity_lower):
                new_cols["parking_off_premises"].at[idx] = True

            # Flag detailed parking types
            for col_name, pattern in parking_patterns.items():
                if re.search(pattern, amenity_lower):
                    new_cols[col_name].at[idx] = True

    # Add columns to df
    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [50]:
def count_parking_types(df):
    parking_columns = [col for col in df.columns if col.startswith("parking_")]
    parking_counts = df[parking_columns].sum().sort_values(ascending=False)
    return parking_counts.reset_index().rename(columns={"index": "parking_type", 0: "count"})

def count_parking_with_and_without_types(df):
    parking_columns = [col for col in df.columns if col.startswith("parking_") and col != "has_parking"]
    has_type = df[parking_columns].any(axis=1)
    with_type = ((df["has_parking"]) & (has_type)).sum()
    without_type = ((df["has_parking"]) & (~has_type)).sum()
    return {"with_type": int(with_type), "without_type": int(without_type)}

In [51]:
df = create_parking_columns(df)

# Get counts
parking_counts_df = count_parking_types(df)
parking_summary = count_parking_with_and_without_types(df)

print(parking_summary)
print(parking_counts_df)


{'with_type': 5234, 'without_type': 0}
                       parking_type  count
0               parking_free_street   2391
1               parking_on_premises   2212
2              parking_off_premises   1908
3                      parking_paid   1860
4               parking_paid_street    901
5                  parking_paid_lot    215
6               parking_paid_garage    207
7             parking_free_driveway     50
8   parking_free_residential_garage     27
9               parking_free_garage     10
10             parking_free_carport      9


In [52]:
# Identify parking columns
parking_columns = [col for col in df.columns if col.startswith("parking_")]

# Get indices where parking detected
indices_with_parking = df[df["has_parking"] == True].index

# Select test index
test_index = 16
if test_index < len(indices_with_parking):
    index_to_check = indices_with_parking[test_index]
    true_parking_columns = [col for col in parking_columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities", "has_parking"] + true_parking_columns
    df_check = df.loc[[index_to_check], columns_to_display]
    
    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected parking features ===")
    print(df_check.to_string(max_colwidth=2000))
else:
    print(f"Only {len(indices_with_parking)} rows with has_parking == True; test_index {test_index} is out of range.")



=== Index: 28 | Displaying amenities and detected parking features ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            amenities  has_parking  parking_on_premises  parking

### Outdoor

In [53]:
def create_outdoor_columns(df):
    # Explicit patterns for detection
    outdoor_patterns = {
        "outdoor_dining_area": r"outdoor dining area",
        "outdoor_furniture": r"outdoor furniture",
        "outdoor_kitchen": r"outdoor kitchen",
        "outdoor_playground": r"outdoor playground",
    }

    # Initialize columns
    new_cols = {
        "has_outdoor": pd.Series(False, index=df.index),
        "outdoor_pool_shared": pd.Series(False, index=df.index),
        "outdoor_pool_private": pd.Series(False, index=df.index),
    }
    for col in outdoor_patterns.keys():
        new_cols[col] = pd.Series(False, index=df.index)

    # Pool patterns
    shared_pool_pattern = re.compile(r"\b(shared outdoor pool|community pool|shared pool)\b")
    private_pool_pattern = re.compile(r"\b(outdoor pool|private outdoor pool|swimming pool)\b")  # Keep "pool" last for broad catch

    # Phrases for has_outdoor detection
    outdoor_terms = [
        "outdoor dining area",
        "outdoor furniture",
        "outdoor kitchen",
        "outdoor playground",
        "outdoor pool",
        "shared outdoor pool",
        "private outdoor pool",
        "community pool",
        "shared pool",
    ]

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        # ✅ Set has_outdoor only if one of the clear outdoor terms is present
        if any(any(term in a for term in outdoor_terms) for a in amenities_lower):
            new_cols["has_outdoor"].at[idx] = True

        # ✅ Check shared/private pools
        for amenity_lower in amenities_lower:
            if shared_pool_pattern.search(amenity_lower):
                new_cols["outdoor_pool_shared"].at[idx] = True
            elif private_pool_pattern.search(amenity_lower):
                # Only mark as private if not marked as shared
                if not shared_pool_pattern.search(amenity_lower):
                    new_cols["outdoor_pool_private"].at[idx] = True

            # ✅ Check specific features
            for col_name, pattern in outdoor_patterns.items():
                if pattern in amenity_lower:
                    new_cols[col_name].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [54]:
def count_outdoor_types(df):
    outdoor_columns = [col for col in df.columns if col.startswith("outdoor_")]
    outdoor_counts = df[outdoor_columns].sum().sort_values(ascending=False)
    return outdoor_counts.reset_index().rename(columns={"index": "outdoor_type", 0: "count"})

def count_outdoor_with_and_without_types(df):
    outdoor_columns = [col for col in df.columns if col.startswith("outdoor_") and col != "has_outdoor"]
    has_type = df[outdoor_columns].any(axis=1)
    with_type = ((df["has_outdoor"]) & (has_type)).sum()
    without_type = ((df["has_outdoor"]) & (~has_type)).sum()
    return {"with_type": int(with_type), "without_type": int(without_type)}


In [55]:
df = create_outdoor_columns(df)

# Get counts
outdoor_counts_df = count_outdoor_types(df)
outdoor_summary = count_outdoor_with_and_without_types(df)

print(outdoor_summary)
print(outdoor_counts_df)


{'with_type': 2097, 'without_type': 0}
           outdoor_type  count
0   outdoor_dining_area   1536
1     outdoor_furniture   1315
2    outdoor_playground    444
3       outdoor_kitchen     32
4   outdoor_pool_shared     22
5  outdoor_pool_private     11


In [56]:
# Identify outdoor columns
outdoor_columns = [col for col in df.columns if col.startswith("outdoor_pool_shared")]

# Get indices where outdoor features detected
indices_with_outdoor = df[df["outdoor_pool_private"] == True].index

# Select test index
test_index = 5
if test_index < len(indices_with_outdoor):
    index_to_check = indices_with_outdoor[test_index]
    true_outdoor_columns = [col for col in outdoor_columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities", "has_outdoor"] + true_outdoor_columns
    df_check = df.loc[[index_to_check], columns_to_display]
    
    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected outdoor features ===")
    print(df_check.to_string(max_colwidth=2000))
else:
    print(f"Only {len(indices_with_outdoor)} rows with has_outdoor == True; test_index {test_index} is out of range.")



=== Index: 5875 | Displaying amenities and detected outdoor features ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

### Utilities

In [57]:
def create_bath_bed_game_columns(df):
    # Mapping of columns to target phrases
    item_patterns = {
        "bath_item_shampoo": "shampoo",
        "bath_item_body_soap": "body soap",
        "bath_item_conditioner": "conditioner",
        "kitchen_item_baking_sheet": "baking sheet",
        "bed_item_bed_linens": "bed linens",
        "game_item_games": "games",
    }

    # Initialize columns
    new_cols = {
        "has_bath_bed_game_items": pd.Series(False, index=df.index)
    }
    for col in item_patterns.keys():
        new_cols[col] = pd.Series(False, index=df.index)

    # For detection efficiency
    item_terms = list(item_patterns.values())

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        # Set general flag if any detected
        if any(any(term in a for term in item_terms) for a in amenities_lower):
            new_cols["has_bath_bed_game_items"].at[idx] = True

        # Set individual columns
        for amenity_lower in amenities_lower:
            for col_name, term in item_patterns.items():
                if term in amenity_lower:
                    new_cols[col_name].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [58]:
def count_bath_bed_game_items(df):
    item_columns = [col for col in df.columns if col.startswith(("bath_item_", "bed_item_", "game_item_", "kitchen_item_"))]
    item_counts = df[item_columns].sum().sort_values(ascending=False)
    return item_counts.reset_index().rename(columns={"index": "item_type", 0: "count"})

def count_bath_bed_game_with_and_without(df):
    item_columns = [col for col in df.columns if col.startswith(("bath_item_", "bed_item_", "game_item_", "kitchen_item_"))]
    has_type = df[item_columns].any(axis=1)
    with_type = ((df["has_bath_bed_game_items"]) & (has_type)).sum()
    without_type = ((df["has_bath_bed_game_items"]) & (~has_type)).sum()
    return {"with_type": int(with_type), "without_type": int(without_type)}


In [59]:
df = create_bath_bed_game_columns(df)

# Get counts
item_counts_df = count_bath_bed_game_items(df)
item_summary = count_bath_bed_game_with_and_without(df)

print(item_summary)
print(item_counts_df)

{'with_type': 7174, 'without_type': 0}
                   item_type  count
0        bed_item_bed_linens   6464
1          bath_item_shampoo   4383
2        bath_item_body_soap   3640
3  kitchen_item_baking_sheet   2826
4      bath_item_conditioner   1103
5            game_item_games    730


In [60]:
# Identify item columns
item_columns = [col for col in df.columns if col.startswith(("bath_item_", "bed_item_", "game_item_", "kitchen_item_"))]

# Get indices where items detected
indices_with_items = df[df["has_bath_bed_game_items"] == True].index

# Select test index
test_index = 7
if test_index < len(indices_with_items):
    index_to_check = indices_with_items[test_index]
    true_item_columns = [col for col in item_columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities", "has_bath_bed_game_items"] + true_item_columns
    df_check = df.loc[[index_to_check], columns_to_display]
    
    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected bath/bed/game items ===")
    print(df_check.to_string(max_colwidth=2000))
else:
    print(f"Only {len(indices_with_items)} rows with has_bath_bed_game_items == True; test_index {test_index} is out of range.")



=== Index: 12 | Displaying amenities and detected bath/bed/game items ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              amenities  has_bath_bed_game_items  kitchen_item_baking_sheet  bed_item_bed_linens
12  ["Dishes and silverware", "Bed linens", "Cleaning available during stay", "Laundromat nearby", "Luggage dropoff allowed", "Long term stays allowed", "Refrigerator", "Hangers", "Wifi", "Dishwasher", "C

### WIFI


In [61]:
import json
import pandas as pd
import re

def create_wifi_columns(df):
    # Initialize columns
    new_cols = {
        "has_wifi": pd.Series(False, index=df.index),
        "wifi_speed_mbps": pd.Series(None, index=df.index, dtype=float)
    }

    # Loosened detection
    wifi_terms_pattern = re.compile(
        r"(wifi|wi-fi|Ethernet|wireless internet)"
    )

    # Improved speed extraction pattern:
    wifi_speed_pattern = re.compile(
        r"(?:wifi.*?(?:–|-)\s*)?(\d{1,4}(?:[.,]\d+)?)\s*(?:mbps|mb/s|mps)"
    )

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        found_wifi = False
        speeds = []

        for amenity in amenities_lower:
            if wifi_terms_pattern.search(amenity):
                found_wifi = True

                match = wifi_speed_pattern.search(amenity)
                if match:
                    speed_str = match.group(1).replace(",", ".")
                    try:
                        speed_value = float(speed_str)
                        speeds.append(speed_value)
                    except ValueError:
                        pass

        if found_wifi:
            new_cols["has_wifi"].at[idx] = True
            if speeds:
                new_cols["wifi_speed_mbps"].at[idx] = max(speeds)

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [62]:
def count_wifi_summary(df):
    has_wifi = df["has_wifi"] == True
    has_speed = df["wifi_speed_mbps"].notna()

    with_speed = (has_wifi & has_speed).sum()
    without_speed = (has_wifi & ~has_speed).sum()
    total_wifi = has_wifi.sum()

    return {
        "total_wifi": int(total_wifi),
        "with_speed": int(with_speed),
        "without_speed": int(without_speed)
    }

def wifi_speed_distribution(df, bins=None):
    wifi_speeds = df.loc[df["wifi_speed_mbps"].notna(), "wifi_speed_mbps"]
    if bins:
        return pd.cut(wifi_speeds, bins=bins).value_counts().sort_index()
    else:
        return wifi_speeds.value_counts().sort_index()


In [63]:
df = create_wifi_columns(df)

# Count summary
wifi_summary = count_wifi_summary(df)
print(wifi_summary)

# Optional: Display raw speeds for review
print(df[["has_wifi", "wifi_speed_mbps"]].head(10))

{'total_wifi': 8659, 'with_speed': 665, 'without_speed': 7994}
    has_wifi  wifi_speed_mbps
0       True              NaN
1       True              NaN
2       True              NaN
3       True              NaN
4       True              NaN
5      False              NaN
7       True              NaN
9       True              NaN
10      True              NaN
11      True              NaN


In [64]:
# Get indices where WiFi detected
indices_with_wifi = df[df["wifi_speed_mbps"].notna()].index

# Select test index
test_index = 102
if test_index < len(indices_with_wifi):
    index_to_check = indices_with_wifi[test_index]
    columns_to_display = ["amenities", "has_wifi", "wifi_speed_mbps"]
    df_check = df.loc[[index_to_check], columns_to_display]

    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected WiFi info ===")
    print(df_check.to_string(max_colwidth=1500))
else:
    print(f"Only {len(indices_with_wifi)} rows with has_wifi == True; test_index {test_index} is out of range.")



=== Index: 2839 | Displaying amenities and detected WiFi info ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       amenities  has_wifi  wifi_speed_mbps
2839  ["Portable fans", "Dishes and silverware", "Garden view", "Bed linens", "TV", "Hangers", "Refrigerator", "Shampoo", "Cleaning products", "Cooking basics", "Essentials", "Coffee maker", "Hair dryer", "Heating", "Hot water kettle", "Self check-in", "Fast wifi \u2013 206 Mbps", "Dedicated workspace", "Lockbox", "Kitchen", "F

### Backyard

In [65]:
def create_backyard_columns(df):
    # Initialize columns
    new_cols = {
        "has_backyard": pd.Series(False, index=df.index),
        "backyard_shared": pd.Series(False, index=df.index),
        "backyard_fully_fenced": pd.Series(False, index=df.index)
    }

    # Detection patterns
    backyard_pattern = re.compile(r"\bbackyard|garten\b")
    shared_pattern = re.compile(r"\b(shared backyard\b)")
    fenced_pattern = re.compile(r"\b(backyard \u2013 fully fenced)\b")

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        found_backyard = False

        for amenity in amenities_lower:
            if backyard_pattern.search(amenity):
                found_backyard = True

                if shared_pattern.search(amenity):
                    new_cols["backyard_shared"].at[idx] = True

                if fenced_pattern.search(amenity):
                    new_cols["backyard_fully_fenced"].at[idx] = True

        if found_backyard:
            new_cols["has_backyard"].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [66]:
def count_backyard_features(df):
    backyard_columns = ["has_backyard", "backyard_shared", "backyard_fully_fenced"]
    backyard_counts = df[backyard_columns].sum().sort_values(ascending=False)
    return backyard_counts.reset_index().rename(columns={"index": "backyard_feature", 0: "count"})


In [67]:
df = create_backyard_columns(df)

# Display counts
backyard_counts_df = count_backyard_features(df)
print(backyard_counts_df)


        backyard_feature  count
0           has_backyard   2100
1        backyard_shared    918
2  backyard_fully_fenced    852


In [68]:
# Identify backyard columns
backyard_columns = ["has_backyard", "backyard_shared", "backyard_fully_fenced"]

# Get indices where backyard features detected
indices_with_backyard = df[df["has_backyard"] == True].index

# Select test index
test_index = 97
if test_index < len(indices_with_backyard):
    index_to_check = indices_with_backyard[test_index]
    true_backyard_columns = [col for col in backyard_columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities"] + true_backyard_columns
    df_check = df.loc[[index_to_check], columns_to_display]

    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected backyard features ===")
    print(df_check.to_string(max_colwidth=1500))
else:
    print(f"Only {len(indices_with_backyard)} rows with has_backyard == True; test_index {test_index} is out of range.")


=== Index: 362 | Displaying amenities and detected backyard features ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

### Balcony

In [69]:
def create_balcony_columns(df):
    # Initialize column
    new_cols = {
        "has_balcony": pd.Series(False, index=df.index)
    }

    # Patterns for "balcony", "balkon", "patio"
    balcony_pattern = re.compile(
        r"\b(balcony|balkon|patio)\b"
    )

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        for amenity in amenities_lower:
            if balcony_pattern.search(amenity):
                new_cols["has_balcony"].at[idx] = True
                break  # no need to check further for this row

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [70]:
def count_balcony(df):
    count = df["has_balcony"].sum()
    return {"total_with_balcony": int(count)}

In [71]:
df = create_balcony_columns(df)

# Count
balcony_summary = count_balcony(df)
print(balcony_summary)

{'total_with_balcony': 2987}


In [72]:
# Get indices with balconies
indices_with_balcony = df[df["has_balcony"] == True].index

# Select test index
test_index = 6
if test_index < len(indices_with_balcony):
    index_to_check = indices_with_balcony[test_index]
    columns_to_display = ["amenities", "has_balcony"]
    df_check = df.loc[[index_to_check], columns_to_display]

    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected balcony ===")
    print(df_check.to_string(max_colwidth=1500))
else:
    print(f"Only {len(indices_with_balcony)} rows with has_balcony == True; test_index {test_index} is out of range.")



=== Index: 21 | Displaying amenities and detected balcony ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

### Heating

In [73]:
def create_heating_columns(df):
    # Initialize columns
    new_cols = {
        "has_heating": pd.Series(False, index=df.index),
        "has_central_heating": pd.Series(False, index=df.index)
    }

    # Heating detection patterns
    heating_pattern = re.compile(r"\b(heating)\b")
    central_heating_pattern = re.compile(r"\b(central heating)\b")

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        for amenity in amenities_lower:
            # Check for central heating
            if central_heating_pattern.search(amenity):
                new_cols["has_central_heating"].at[idx] = True

            # Check for any heating mention
            if heating_pattern.search(amenity):
                new_cols["has_heating"].at[idx] = True

            # If central heating detected, also ensure has_heating is True
            if new_cols["has_central_heating"].at[idx]:
                new_cols["has_heating"].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [74]:
def count_heating_features(df):
    heating_columns = ["has_heating", "has_central_heating"]
    heating_counts = df[heating_columns].sum().sort_values(ascending=False)
    return heating_counts.reset_index().rename(columns={"index": "heating_feature", 0: "count"})


In [75]:
df = create_heating_columns(df)

# Display counts
heating_counts_df = count_heating_features(df)
print(heating_counts_df)

       heating_feature  count
0          has_heating   7703
1  has_central_heating   2487


In [76]:
# Identify heating columns
heating_columns = ["has_heating", "has_central_heating"]

# Get indices where heating is detected
indices_with_heating = df[df["has_heating"] == True].index

# Select test index
test_index = 7
if test_index < len(indices_with_heating):
    index_to_check = indices_with_heating[test_index]
    true_heating_columns = [col for col in heating_columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities"] + true_heating_columns
    df_check = df.loc[[index_to_check], columns_to_display]

    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected heating features ===")
    print(df_check.to_string(max_colwidth=1500))
else:
    print(f"Only {len(indices_with_heating)} rows with has_heating == True; test_index {test_index} is out of range.")



=== Index: 9 | Displaying amenities and detected heating features ===
                                                                                                                                                                                                      amenities  has_heating
9  ["Heating", "Host greets you", "Smoke alarm", "Coffee maker", "Dishes and silverware", "Kitchen", "Private entrance", "Ethernet connection", "Refrigerator", "TV", "Hangers", "Washer", "Wifi", "Hot water"]         True


### Hottub and Bathtub

In [77]:
def create_hottub_bathtub_columns(df):
    # Initialize columns
    new_cols = {
        "has_hottub": pd.Series(False, index=df.index),
        "has_bathtub": pd.Series(False, index=df.index)
    }

    # Patterns
    hottub_pattern = re.compile(r"\b(hot tub)\b")
    bathtub_pattern = re.compile(r"\b(bathtub)\b")

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        for amenity in amenities_lower:
            # Check for hot tub
            if hottub_pattern.search(amenity):
                new_cols["has_hottub"].at[idx] = True

            # Check for bathtub
            if bathtub_pattern.search(amenity):
                new_cols["has_bathtub"].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [78]:
def count_hottub_bathtub(df):
    columns = ["has_hottub", "has_bathtub"]
    counts = df[columns].sum().sort_values(ascending=False)
    return counts.reset_index().rename(columns={"index": "feature", 0: "count"})

In [79]:
df = create_hottub_bathtub_columns(df)

# Get counts
ht_bt_counts_df = count_hottub_bathtub(df)
print(ht_bt_counts_df)

       feature  count
0  has_bathtub   2692
1   has_hottub    135


In [80]:
# Identify relevant columns
columns = ["has_hottub", "has_bathtub"]

# Get indices where hot tub or bathtub detected
indices_with_features = df[(df["has_hottub"] == True) ].index

# Select test index
test_index = 7
if test_index < len(indices_with_features):
    index_to_check = indices_with_features[test_index]
    true_columns = [col for col in columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities"] + true_columns
    df_check = df.loc[[index_to_check], columns_to_display]

    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected hot tub / bathtub features ===")
    print(df_check.to_string(max_colwidth=1500))
else:
    print(f"Only {len(indices_with_features)} rows with hot tub or bathtub detected; test_index {test_index} is out of range.")



=== Index: 910 | Displaying amenities and detected hot tub / bathtub features ===
                                                                                                                            amenities  has_hottub
910  ["Heating", "Dryer", "Elevator", "Kitchen", "Pets allowed", "Wifi", "Essentials", "Hot tub", "Washer", "TV with standard cable"]        True


### Washer and dryer

In [81]:
def create_washer_dryer_columns(df):
    # Initialize columns
    new_cols = {
        "has_washer": pd.Series(False, index=df.index),
        "washer_paid": pd.Series(False, index=df.index),
        "washer_in_building": pd.Series(False, index=df.index),
        "has_dryer": pd.Series(False, index=df.index),
        "dryer_paid": pd.Series(False, index=df.index),
        "dryer_in_building": pd.Series(False, index=df.index),
    }

    # Patterns
    washer_pattern = re.compile(r"\b(washer)\b")
    dryer_pattern = re.compile(r"\b(dryer)\b")
    paid_pattern = re.compile(r"\b(paid)\b")
    in_building_pattern = re.compile(r"\b(in building|on premises)\b")

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        for amenity in amenities_lower:
            # Washer detection
            if washer_pattern.search(amenity):
                new_cols["has_washer"].at[idx] = True
                if paid_pattern.search(amenity):
                    new_cols["washer_paid"].at[idx] = True
                if in_building_pattern.search(amenity):
                    new_cols["washer_in_building"].at[idx] = True

            # Dryer detection
            if dryer_pattern.search(amenity):
                new_cols["has_dryer"].at[idx] = True
                if paid_pattern.search(amenity):
                    new_cols["dryer_paid"].at[idx] = True
                if in_building_pattern.search(amenity):
                    new_cols["dryer_in_building"].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [82]:
def count_washer_dryer_features(df):
    columns = [
        "has_washer", "washer_paid", "washer_in_building",
        "has_dryer", "dryer_paid", "dryer_in_building"
    ]
    counts = df[columns].sum().sort_values(ascending=False)
    return counts.reset_index().rename(columns={"index": "feature", 0: "count"})

In [83]:
df = create_washer_dryer_columns(df)

# Get counts
washer_dryer_counts_df = count_washer_dryer_features(df)
print(washer_dryer_counts_df)


              feature  count
0           has_dryer   7075
1          has_washer   6730
2  washer_in_building    892
3   dryer_in_building    583
4         washer_paid    441
5          dryer_paid    275


In [84]:
# Identify columns
columns = [
    "has_washer", "washer_paid", "washer_in_building",
    "has_dryer", "dryer_paid", "dryer_in_building"
]

# Get indices where washer or dryer detected
indices_with_features = df[
    (df["has_washer"] == True) & (df["dryer_paid"] == True)
].index

# Select test index
test_index = 17
if test_index < len(indices_with_features):
    index_to_check = indices_with_features[test_index]
    true_columns = [col for col in columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities"] + true_columns
    df_check = df.loc[[index_to_check], columns_to_display]

    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected washer/dryer features ===")
    print(df_check.to_string(max_colwidth=2500))
else:
    print(f"Only {len(indices_with_features)} rows with washer or dryer detected; test_index {test_index} is out of range.")



=== Index: 3606 | Displaying amenities and detected washer/dryer features ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    amenities  has_washer  washer_paid  washer_in_building  has_dryer  dryer_paid  dryer_in_building
3606

### Hoesekeeping

In [85]:
def create_housekeeping_columns(df):
    # Initialize columns
    new_cols = {
        "housekeeping_included": pd.Series(False, index=df.index),
        "housekeeping_available_at_cost": pd.Series(False, index=df.index)
    }

    housekeeping_pattern = re.compile(
        r"\b(housekeeping)\b"
    )
    

  

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        for amenity in amenities_lower:
            # Check for housekeeping included
            if housekeeping_pattern.search(amenity):
                if("cost" in amenity):
                    new_cols["housekeeping_available_at_cost"].at[idx] = True
                else:
                    new_cols["housekeeping_included"].at[idx] = True
                    
                
                

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [86]:
def count_housekeeping_features(df):
    housekeeping_columns = ["housekeeping_included", "housekeeping_available_at_cost"]
    counts = df[housekeeping_columns].sum().sort_values(ascending=False)
    return counts.reset_index().rename(columns={"index": "housekeeping_feature", 0: "count"})


In [87]:
df = create_housekeeping_columns(df)

# Get counts
housekeeping_counts_df = count_housekeeping_features(df)
print(housekeeping_counts_df)

             housekeeping_feature  count
0  housekeeping_available_at_cost    325
1           housekeeping_included     35


In [88]:
# Identify columns
housekeeping_columns = ["housekeeping_included", "housekeeping_available_at_cost"]

# Get indices where housekeeping detected
indices_with_housekeeping = df[
    (df["housekeeping_included"] == True) | (df["housekeeping_available_at_cost"] == True)
].index

# Select test index
test_index = 3
if test_index < len(indices_with_housekeeping):
    index_to_check = indices_with_housekeeping[test_index]
    true_columns = [col for col in housekeeping_columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities"] + true_columns
    df_check = df.loc[[index_to_check], columns_to_display]

    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected housekeeping features ===")
    print(df_check.to_string(max_colwidth=2500))
else:
    print(f"Only {len(indices_with_housekeeping)} rows with housekeeping detected; test_index {test_index} is out of range.")



=== Index: 475 | Displaying amenities and detected housekeeping features ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           amenities  hous

### Baby and children

In [89]:
def create_baby_columns(df):
    # Mapping of columns to detection phrases
    baby_patterns = {
        "baby_bath": r"\bbaby bath\b",
        "baby_monitor": r"\bbaby monitor\b",
        "baby_safety": r"\b(baby safety|child safety|baby gate|safety gate|cabinet locks)\b",
        "children_books_and_toys": r"\b(children's books and toys|children books and toys|kids books and toys|toys|books for kids|books and toys)\b",
        "crib": r"\b(crib|baby bed|cot)\b",
        "high_chair": r"\b(high chair|baby chair)\b",
    }

    # Initialize columns
    new_cols = {}
    for col in baby_patterns.keys():
        new_cols[col] = pd.Series(False, index=df.index)

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        for amenity_lower in amenities_lower:
            for col_name, pattern in baby_patterns.items():
                if re.search(pattern, amenity_lower):
                    new_cols[col_name].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [90]:
def count_baby_features(df):
    baby_columns = [
        "baby_bath",
        "baby_monitor",
        "baby_safety",
        "children_books_and_toys",
        "crib",
        "high_chair"
    ]
    counts = df[baby_columns].sum().sort_values(ascending=False)
    return counts.reset_index().rename(columns={"index": "baby_feature", 0: "count"})


In [91]:
df = create_baby_columns(df)

# Get counts
baby_counts_df = count_baby_features(df)
print(baby_counts_df)


              baby_feature  count
0                     crib   2442
1               high_chair   1721
2  children_books_and_toys    802
3                baby_bath    355
4             baby_monitor     76
5              baby_safety     58


In [92]:
# Identify baby columns
baby_columns = [
    "baby_bath",
    "baby_monitor",
    "baby_safety",
    "children_books_and_toys",
    "crib",
    "high_chair"
]

# Get indices where any baby feature is detected
indices_with_baby = df[df[baby_columns].any(axis=1)].index

# Select test index
test_index = 12
if test_index < len(indices_with_baby):
    index_to_check = indices_with_baby[test_index]
    true_columns = [col for col in baby_columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities"] + true_columns
    df_check = df.loc[[index_to_check], columns_to_display]

    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected baby features ===")
    print(df_check.to_string(max_colwidth=1500))
else:
    print(f"Only {len(indices_with_baby)} rows with baby features detected; test_index {test_index} is out of range.")



=== Index: 29 | Displaying amenities and detected baby features ===
                                                                                                                                                                                                                                                                amenities  crib  high_chair
29  ["Heating", "Host greets you", "Smoke alarm", "High chair", "Crib", "Kitchen", "Iron", "Luggage dropoff allowed", "Refrigerator", "Free street parking", "Long term stays allowed", "Essentials", "Hangers", "Coffee maker", "Wifi", "TV", "Hair dryer", "Hot water"]  True        True


### Variuos

In [93]:
import json
import pandas as pd
import re

def create_miscellaneous_columns(df):
    # Mapping of column names to detection patterns
    misc_patterns = {
        "luggage_dropoff_allowed": r"\bluggage dropoff allowed\b",
        "long_term_stays_allowed": r"\blong term stays allowed\b",
        "air_conditioning": r"\bair conditioning\b",
        "pets_allowed": r"\b(pets allowed|pet friendly)\b",
        "ping_pong_table": r"\bping pong table\b",
        "host_greets_you": r"\bhost greets you\b",
        "pool_table": r"\bpool table\b",
        "private_entrance": r"\bprivate entrance\b",
        "workspace": r"\b(workspace|dedicated workspace|desk|working space|work space)\b"
    }

    # Initialize columns
    new_cols = {}
    for col in misc_patterns.keys():
        new_cols[col] = pd.Series(False, index=df.index)

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        for amenity_lower in amenities_lower:
            for col_name, pattern in misc_patterns.items():
                if re.search(pattern, amenity_lower):
                    new_cols[col_name].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [94]:
def count_miscellaneous_features(df):
    misc_columns = [
        "luggage_dropoff_allowed",
        "long_term_stays_allowed",
        "air_conditioning",
        "pets_allowed",
        "ping_pong_table",
        "host_greets_you",
        "private_entrance",
        "workspace"
    ]
    counts = df[misc_columns].sum().sort_values(ascending=False)
    return counts.reset_index().rename(columns={"index": "misc_feature", 0: "count"})


In [95]:
df = create_miscellaneous_columns(df)

# Display counts
misc_counts_df = count_miscellaneous_features(df)
print(misc_counts_df)


              misc_feature  count
0                workspace   5344
1  long_term_stays_allowed   4156
2  luggage_dropoff_allowed   2531
3         private_entrance   2514
4          host_greets_you   2128
5             pets_allowed   2066
6         air_conditioning    631
7          ping_pong_table    144


In [96]:
# Identify miscellaneous columns
misc_columns = [
    "luggage_dropoff_allowed",
    "long_term_stays_allowed",
    "air_conditioning",
    "pets_allowed",
    "ping_pong_table",
    "host_greets_you",
    "private_entrance",
    "workspace"
]

# Get indices where any miscellaneous feature is detected
indices_with_misc = df[df[misc_columns].any(axis=1)].index

# Select test index
test_index = 80
if test_index < len(indices_with_misc):
    index_to_check = indices_with_misc[test_index]
    true_columns = [col for col in misc_columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities"] + true_columns
    df_check = df.loc[[index_to_check], columns_to_display]

    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected miscellaneous features ===")
    print(df_check.to_string(max_colwidth=1500))
else:
    print(f"Only {len(indices_with_misc)} rows with miscellaneous features detected; test_index {test_index} is out of range.")



=== Index: 108 | Displaying amenities and detected miscellaneous features ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  amenities  luggage_dropoff_allowed  long_term_stays_allowed  pets_allowed
108  ["Bathtub", "Drying rack for clothing", "Dishes and silverware", "Bed linens", "Luggage dropoff allowed", "Long term stays allowed", "TV

### View

In [97]:
def create_view_columns(df):
    # Mapping columns to detection patterns
    view_patterns = {
        "beach_view": r"\bbeach view\b",
        "city_skyline_view": r"\bcity skyline view\b",
        "desert_view": r"\bdesert view\b",
        "garden_view": r"\bgarden view\b",
        "pool_view": r"\bpool view\b",

    }

    # Initialize columns
    new_cols = {}
    for col in view_patterns.keys():
        new_cols[col] = pd.Series(False, index=df.index)

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        for amenity_lower in amenities_lower:
            for col_name, pattern in view_patterns.items():
                if re.search(pattern, amenity_lower):
                    new_cols[col_name].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df


In [98]:
def count_view_features(df):
    view_columns = ["beach_view", "city_skyline_view", "desert_view", "garden_view","pool_view"]
    counts = df[view_columns].sum().sort_values(ascending=False)
    return counts.reset_index().rename(columns={"index": "view_feature", 0: "count"})


In [99]:
df = create_view_columns(df)

# Display counts
view_counts_df = count_view_features(df)
print(view_counts_df)

        view_feature  count
0        garden_view    557
1  city_skyline_view    411
2          pool_view     14
3         beach_view      6
4        desert_view      1


In [100]:
view_columns = ["beach_view", "city_skyline_view", "desert_view", "garden_view","pool_view"]

# Get indices where any miscellaneous feature is detected
indices_with_misc = df[df[view_columns].any(axis=1)].index

# Select test index
test_index = 22
if test_index < len(indices_with_misc):
    index_to_check = indices_with_misc[test_index]
    true_columns = [col for col in view_columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities"] + true_columns
    df_check = df.loc[[index_to_check], columns_to_display]

    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected miscellaneous features ===")
    print(df_check.to_string(max_colwidth=1500))
else:
    print(f"Only {len(indices_with_misc)} rows with miscellaneous features detected; test_index {test_index} is out of range.")



=== Index: 166 | Displaying amenities and detected miscellaneous features ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     amenities  city_skyline_view
166  ["Dryer", "Drying rack for clothing", "Dishes and silverware", "City skyline view", "Bed linens", "Laundromat nearby", "Outdoor dining area", "Refrigerator", "Coffee", "Wifi", "Dishwasher", "32 inch TV with Amazon Prime Video", "Cooking basics", "Fire extinguisher", "Stainless steel oven", "Essentials", "

### Gym 

In [101]:
def create_gym_columns(df):
    # Mapping columns to detection patterns
    gym_patterns = {
        "gym_private_in_building": r"\bprivate gym in building\b",
        "gym_private_nearby": r"\bprivate gym nearby\b",
        "gym_shared_in_building": r"\bshared gym in building\b",
        "gym_shared_nearby": r"\bshared gym nearby\b",
        "gym_private": r"\bprivate gym\b",
        "gym_shared": r"\bshared gym\b",
    }

    # Initialize columns
    new_cols = {}
    for col in gym_patterns.keys():
        new_cols[col] = pd.Series(False, index=df.index)

    # Optional general gym detection
    new_cols["gym"] = pd.Series(False, index=df.index)

    for idx in df.index:
        amenities = get_amenities_at_index(df, idx)
        amenities_lower = [a.lower() for a in amenities]

        for amenity_lower in amenities_lower:
            # Mark specific private/shared patterns
            for col_name, pattern in gym_patterns.items():
                if re.search(pattern, amenity_lower):
                    new_cols[col_name].at[idx] = True

            # General gym detection
            if "gym" in amenity_lower:
                new_cols["gym"].at[idx] = True

    df = pd.concat([df, pd.DataFrame(new_cols)], axis=1)
    return df

In [102]:
def count_gym_features(df):
    gym_columns = [
        "gym",
        "gym_private",
        "gym_private_in_building",
        "gym_private_nearby",
        "gym_shared",
        "gym_shared_in_building",
        "gym_shared_nearby"
    ]
    counts = df[gym_columns].sum().sort_values(ascending=False)
    return counts.reset_index().rename(columns={"index": "gym_feature", 0: "count"})

In [103]:
df = create_gym_columns(df)

# Display counts
gym_counts_df = count_gym_features(df)
print(gym_counts_df)

               gym_feature  count
0                      gym    179
1               gym_shared     87
2   gym_shared_in_building     53
3        gym_shared_nearby     32
4              gym_private     25
5  gym_private_in_building     22
6       gym_private_nearby      1


In [104]:
# Identify gym columns
gym_columns = [
    "gym",
    "gym_private",
    "gym_private_in_building",
    "gym_private_nearby",
    "gym_shared",
    "gym_shared_in_building",
    "gym_shared_nearby"
]

# Get indices where any gym feature is detected
indices_with_gym = df[df[gym_columns].any(axis=1)].index

# Select test index
test_index = 14
if test_index < len(indices_with_gym):
    index_to_check = indices_with_gym[test_index]
    true_columns = [col for col in gym_columns if df.at[index_to_check, col] == True]
    columns_to_display = ["amenities"] + true_columns
    df_check = df.loc[[index_to_check], columns_to_display]

    print(f"\n=== Index: {index_to_check} | Displaying amenities and detected gym features ===")
    print(df_check.to_string(max_colwidth=1500))
else:
    print(f"Only {len(indices_with_gym)} rows with gym features detected; test_index {test_index} is out of range.")


=== Index: 2431 | Displaying amenities and detected gym features ===
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               amenities   gym  gym_shared  gym_shared_in_building
2431  ["Shared backyard \u2013 Fully fenced", "Portable fans", "Shower gel", "Dishes and silverware", "Laundromat nearby", "Long term stays allowed", "Ping pong table", "Refrigerator", "Hangers", "TV", "EV charger", "Paid parking on premises", "Cooking basics", "Essentials", "Clothing s

In [105]:
df.shape

(8898, 260)

### Square meters


In [106]:
# Function to extract size in square meters
def extract_square_meters(text):
    if pd.isnull(text):
        return None
    match = re.search(r"\b(\d{1,3})\s?(m2|m²|sqm|square meter|qm|quadratmeter)\b", text.lower())
    if match:
        return int(match.group(1))
    return None

# Apply to both 'name' and 'description', preferring 'name' first if both are present
def get_size_from_name_or_description(row):
    name_size = extract_square_meters(row["name"])
    if name_size:
        return name_size
    return extract_square_meters(row["description"])

In [107]:
df["sqm"] = df.apply(get_size_from_name_or_description, axis=1)

In [108]:
df_check = df[df["sqm"].notna()]
df_check = df_check[["name","description","sqm"]]
# Select test index
test_index = 15

print(df_check[test_index:test_index+1].to_string(max_colwidth=2500))


                                  name                                                                                                                                                                                                                                                                                                                                                                                                                              description    sqm
137  LOVINGLY RESTORED 100qm APARTMENT  Our lovingly restored pre-war apartment in the famouse Kollwitzstraße, Berlin Prenzlauer Berg, holds up to 4 people in two separate bedrooms. Our apartment is located in the popular Kollwitzstrasse, one of the most beautiful areas of Berlin. Children are welcome-  while parties and smoking are not permitted. A four day minimum stay is required. For questions on availability, we would appreciate a short email from you...  100.0


### Fill with name and description

In [109]:
# import pandas as pd
# import re

# def fill_features_from_name_and_description(df):
#     # Patterns for each column (EN + DE)
#     feature_patterns = {
#         "has_backyard": r"\b(backyard|garten|hof)\b",
#         "backyard_shared": r"\b(shared backyard|gemeinschaftsgarten)\b",
#         "backyard_fully_fenced": r"\b(fully fenced backyard|eingezäunter garten|komplett umzäunt)\b",
#         "has_balcony": r"\b(balcony|balkon)\b",
#         "has_hottub": r"\b(hot tub|whirlpool|jacuzzi)\b",
#         "private_entrance": r"\b(private entrance|eigener eingang)\b",
#         "shared_entrance": r"\b(shared entrance|geteilten eingang|geteilter eingang)\b",
#         "workspace": r"\b(workspace|arbeitsplatz|schreibtisch|arbeitsbereich)\b",
#         "outdoor_pool_shared": r"\b(shared outdoor pool|gemeinschaftspool)\b",
#         "outdoor_pool_private": r"\b(private outdoor pool|eigener pool|privatpool)\b",
#     }

#     for feature, pattern in feature_patterns.items():
#         regex = re.compile(pattern, flags=re.IGNORECASE)
#         df[feature] = df.apply(
#             lambda row: bool(regex.search(
#                 f"{str(row.get('name', ''))} {str(row.get('description', ''))}"
#             )),
#             axis=1
#         )

#     return df


In [110]:
#df = fill_features_from_name_and_description(df)

#### Distance to center 

In [111]:
import numpy as np

# Define Berlin center coordinates
berlin_lat = 52.5200
berlin_lon = 13.4050

# Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2.0) ** 2 + \
        np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2

    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Compute distance to Berlin center and create the new column
df['distance_to_center'] = haversine(df['latitude'], df['longitude'], berlin_lat, berlin_lon)
# Select relevant columns
display_cols = ['longitude', 'latitude', 'distance_to_center']

# Sort by distance_to_center
df_sorted = df[display_cols].sort_values(by='distance_to_center', ascending=True)

# Display cleanly
print(df_sorted[0:5])

       longitude  latitude  distance_to_center
7074    13.40584  52.51937            0.090209
6361    13.40517  52.52113            0.126176
1426    13.40392  52.51898            0.134921
8733    13.40514  52.52126            0.140425
13018   13.40516  52.52139            0.154940


#### Distance to ubahn stations

In [112]:
# import requests
# from bs4 import BeautifulSoup
# import re
# from geopy.distance import geodesic

# # Function to convert DMS to decimal degrees
# def dms_to_decimal(degrees, minutes, seconds, direction):
#     decimal = float(degrees) + float(minutes) / 60 + float(seconds) / 3600
#     if direction in ['S', 'W']:  # South and West should be negative
#         decimal = -decimal
#     return decimal

# # URL of the Wikipedia page
# url = 'https://de.wikipedia.org/wiki/Liste_der_Berliner_U-Bahnh%C3%B6fe'

# # Fetch the page content
# response = requests.get(url)
# soup = BeautifulSoup(response.text, 'html.parser')

# # Get the entire text of the page
# page_text = soup.get_text()

# # Define a regular expression pattern for coordinates
# coordinate_pattern = r'(\d{1,3})°\s(\d{1,2})′\s(\d{1,2})″\s([NS]),\s(\d{1,3})°\s(\d{1,2})′\s(\d{1,2})″\s([EO])'

# # Find all matches in the page text
# coordinates = re.findall(coordinate_pattern, page_text)

# list_coordinates = []

# # Convert the found coordinates to decimal degrees and print them
# for coord in coordinates:
#     # Latitude
#     lat_deg, lat_min, lat_sec, lat_dir = coord[0], coord[1], coord[2], coord[3]
#     lat_decimal = dms_to_decimal(lat_deg, lat_min, lat_sec, lat_dir)
    
#     # Longitude
#     lon_deg, lon_min, lon_sec, lon_dir = coord[4], coord[5], coord[6], coord[7]
    
#     lon_decimal = dms_to_decimal(lon_deg, lon_min, lon_sec, lon_dir)
#     list_coordinates.append((lat_decimal,lon_decimal))
#     # Print the results
#     #print(f"Latitude: {lat_decimal}, Longitude: {lon_decimal}")


In [113]:
# def minimum_distance_to_ubahn(row):
#     # Get the coordinates of the listing
#     listing_coords = (row['latitude'], row['longitude'])

    
#     # Calculate the distance to each U-Bahn station
#     distances = [geodesic(listing_coords, ubahn).kilometers for ubahn in list_coordinates]
    
#     # Return the minimum distance
#     return min(distances)

In [114]:
# This takes a while
# df["distance_ubahn"] = df.apply(minimum_distance_to_ubahn, axis=1)

In [115]:
# print(df[["latitude", "longitude", "distance_ubahn"]].sort_values(by="distance_ubahn", ascending=True).head(20))

#### Host lives in Berlin

In [116]:
df["host_in_berlin"] = df["host_location"].astype(str).str.contains("berlin", case=False, na=False)
df["host_in_berlin"]

0        False
1         True
2         True
3        False
4        False
         ...  
13940    False
13941    False
13942     True
13943    False
13944     True
Name: host_in_berlin, Length: 8898, dtype: bool

#### Make host neighbourhood numeric

In [117]:
neighbourhood_coords = {
    "Prenzlauer Berg": (13.424, 52.538),
    "Kreuzberg": (13.403, 52.499),
    "Neukölln": (13.438, 52.480),
    "Charlottenburg": (13.291, 52.505),
    "Friedrichshain": (13.454, 52.515),
    "Mitte": (13.400, 52.520),
    "Wedding": (13.363, 52.552),
    "Schöneberg": (13.356, 52.483),
    "Tiergarten": (13.353, 52.516),
    "Wilmersdorf": (13.308, 52.487),
    "Moabit": (13.339, 52.530),
    "Tempelhof": (13.385, 52.468),
    "Lichtenberg": (13.499, 52.515),
    "Treptow": (13.471, 52.480),
    "Steglitz": (13.326, 52.456),
    "Marzahn": (13.545, 52.545),
    "Pankow": (13.401, 52.569),
    "Reinickendorf": (13.334, 52.584),
    "Spandau": (13.206, 52.535),
}
# Function to map neighbourhood to coordinates
def get_neighbourhood_coords(neighbourhood):
    if pd.isna(neighbourhood):
        return pd.Series({"longitude_neighbourhood": None, "latitude_neighbourhood": None})
    coords = neighbourhood_coords.get(neighbourhood.strip())
    if coords:
        lon, lat = coords
        return pd.Series({"longitude_neighbourhood": lon, "latitude_neighbourhood": lat})
    else:
        return pd.Series({"longitude_neighbourhood": None, "latitude_neighbourhood": None})

# Apply to df
df[["longitude_host_neighbourhood", "latitude_host_neighbourhood"]] = df["host_neighbourhood"].apply(get_neighbourhood_coords)

In [118]:
print(df[["host_neighbourhood", "longitude_host_neighbourhood", "longitude_host_neighbourhood"]].head(10))


   host_neighbourhood  longitude_host_neighbourhood  \
0     Prenzlauer Berg                        13.424   
1     Prenzlauer Berg                        13.424   
2     Prenzlauer Berg                        13.424   
3           Kreuzberg                        13.403   
4          Copacabana                           NaN   
5           Kreuzberg                        13.403   
7               Mitte                        13.400   
9     Prenzlauer Berg                        13.424   
10    Prenzlauer Berg                        13.424   
11        Wilmersdorf                        13.308   

    longitude_host_neighbourhood  
0                         13.424  
1                         13.424  
2                         13.424  
3                         13.403  
4                            NaN  
5                         13.403  
7                         13.400  
9                         13.424  
10                        13.424  
11                        13.308  


#### Make host response time numeric 

In [119]:
import numpy as np

# Define mapping dictionary
response_time_mapping = {
    "within an hour": 1,
    "within a few hours": 3,
    "within a day": 24,
    "within a few days": 72,
    "a few days or more": 96
}

# Clean, lower, and map
df["host_response_time_numeric"] = (
    df["host_response_time"]
    .astype(str)
    .str.strip()
    .str.lower()
    .map(response_time_mapping)
)

# Convert 'object' dtype to numeric explicitly
df["host_response_time_numeric"] = pd.to_numeric(df["host_response_time_numeric"], errors='coerce')
print(df[["host_response_time", "host_response_time_numeric"]].dropna().head(10))

    host_response_time  host_response_time_numeric
0         within a day                        24.0
1   a few days or more                        96.0
2   within a few hours                         3.0
4       within an hour                         1.0
7   within a few hours                         3.0
9   within a few hours                         3.0
10      within an hour                         1.0
11  within a few hours                         3.0
16      within an hour                         1.0
17      within an hour                         1.0


In [120]:
#### Make host response rate, host acceptence rate numeric

In [121]:
# Remove '%' and convert to numeric
df["host_response_rate_numeric"] = (
    df["host_response_rate"]
    .astype(str)
    .str.strip()
    .str.replace("%", "", regex=False)
    .replace("nan", np.nan)  # in case of 'nan' strings
    .astype(float)
)

df["host_acceptance_rate_numeric"] = (
    df["host_acceptance_rate"]
    .astype(str)
    .str.strip()
    .str.replace("%", "", regex=False)
    .replace("nan", np.nan)
    .astype(float)
)

### Remove unused columns

In [122]:
df.shape

(8898, 268)

In [123]:
for c in df.columns:
    print(c+" ",end="")

id listing_url scrape_id last_scraped source name description neighborhood_overview picture_url host_id host_url host_name host_since host_location host_about host_response_time host_response_rate host_acceptance_rate host_is_superhost host_thumbnail_url host_picture_url host_neighbourhood host_listings_count host_total_listings_count host_verifications host_has_profile_pic host_identity_verified neighbourhood neighbourhood_cleansed neighbourhood_group_cleansed latitude longitude property_type room_type accommodates bathrooms bathrooms_text bedrooms beds amenities price minimum_nights maximum_nights minimum_minimum_nights maximum_minimum_nights minimum_maximum_nights maximum_maximum_nights minimum_nights_avg_ntm maximum_nights_avg_ntm calendar_updated has_availability availability_30 availability_60 availability_90 availability_365 calendar_last_scraped number_of_reviews number_of_reviews_ltm number_of_reviews_l30d availability_eoy number_of_reviews_ly estimated_occupancy_l365d estimat

In [124]:
columns_to_remove = [

    # Cheating
    "estimated_revenue_l365d",  
    
    # We have these transformed/in some other variable
    "host_response_time","host_response_rate","amenities","description","host_location","host_neighbourhood","neighborhood_overview", "neighbourhood","bathrooms_text","name","host_about",

    
    "host_total_listings_count ","calculated_host_listings_count_entire_homes",
    "number_of_reviews_ly","number_of_reviews_ltm","first_review", "last_review",

    "minimum_minimum_nights", "maximum_maximum_nights","maximum_nights_avg_ntm","maximum_maximum_nights","minimum_maximum_nights ","maximum_minimum_nights","minimum_maximum_nights","minimum_nights_avg_ntm",
    "availability_60","availability_90","availability_365","availability_eoy",
    

    # Very likely not relevant/ allready have
    "license","host_name", "host_since","host_has_profile_pic","has_availability",  "host_verifications",    "calendar_updated",      
    
    # Definitly not relevant

    # URLS
    "listing_url", "picture_url", "host_thumbnail_url", "host_picture_url","host_url",
    
    # Scraping stuff
    "scrape_id", "last_scraped", "source", "calendar_last_scraped","scrape_id",
    # IDS
     "host_id" 
]

In [125]:
# "instant_bookable","host_is_superhost","host_acceptance_rate","host_identity_verified"

In [126]:
df = df.drop(columns=[col for col in columns_to_remove if col in df.columns])

In [127]:
df.shape

(8898, 224)

### Convert Text/Object to categorical

In [128]:
# Count and print the number of columns by type
bool_cols = df.select_dtypes(include='bool').columns.tolist()
cat_cols = df.select_dtypes(include=['category']).columns.tolist()
str_cols = df.select_dtypes(include=['object', 'string']).columns.tolist()
num_cols = df.select_dtypes(include=['int', 'float']).columns.tolist()
other_cols = [col for col in df.columns if col not in bool_cols + cat_cols + str_cols + num_cols]

print(f"🟩 Boolean columns: {len(bool_cols)}")
print(f"🟦 Categorical columns: {len(cat_cols)}")
print(f"🟧 Object/String columns: {len(str_cols)}")
print(f"🟨 Numeric columns: {len(num_cols)}")
print(f"🟥 Other columns: {len(other_cols)}")

🟩 Boolean columns: 180
🟦 Categorical columns: 0
🟧 Object/String columns: 8
🟨 Numeric columns: 36
🟥 Other columns: 0


In [129]:
# Assuming df is your DataFrame
cols_to_convert = ["instant_bookable", "host_is_superhost", "host_acceptance_rate", "host_identity_verified"]

# Convert 'true'/'false' strings to actual booleans
df[cols_to_convert] = df[cols_to_convert].apply(lambda col: col.str.lower() == 't')

In [130]:
# Count and print the number of columns by type
bool_cols = df.select_dtypes(include='bool').columns.tolist()
cat_cols = df.select_dtypes(include=['category']).columns.tolist()
str_cols = df.select_dtypes(include=['object', 'string']).columns.tolist()
num_cols = df.select_dtypes(include=['int', 'float']).columns.tolist()
other_cols = [col for col in df.columns if col not in bool_cols + cat_cols + str_cols + num_cols]

print(f"🟩 Boolean columns: {len(bool_cols)}")
print(f"🟦 Categorical columns: {len(cat_cols)}")
print(f"🟧 Object/String columns: {len(str_cols)}")
print(f"🟨 Numeric columns: {len(num_cols)}")
print(f"🟥 Other columns: {len(other_cols)}")

🟩 Boolean columns: 184
🟦 Categorical columns: 0
🟧 Object/String columns: 4
🟨 Numeric columns: 36
🟥 Other columns: 0


### Drop where no Review,bathroom,bedroom

In [131]:
print(df.shape)
df = df[df["review_scores_rating"].notna()]
df = df[df["bathrooms"].notna()]
df = df[df["bedrooms"].notna()]
print(df.shape)

(8898, 224)
(6845, 224)


### Drop IDS by hand

In [132]:
# Note this does not infer Knowledge

In [133]:
# List of IDs to drop
ids_to_drop = [
    2860420,
    7807397,
    41787739,
    928290731641806731,
    967963843443094544,
    1234474294833419521,
    1234483190089885723,
    28432988,
    559822192883775161,
    559824222713920928,
    6663931,
    29695826,
    1234483190089885723, # added
    20109066,
    38585015,
    967963843443094544
]

# Drop rows with these IDs
df = df[~df['id'].isin(ids_to_drop)].copy()
print(df.shape)
print(df.shape)





(6831, 224)
(6831, 224)


### Split Data

In [134]:

# Split into train and test
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Confirm split
print(f"Train set size: {df_train.shape[0]} rows")
print(f"Test set size: {df_test.shape[0]} rows")

Train set size: 5464 rows
Test set size: 1367 rows


In [135]:
df_train_id = df_train['id']
df_test_id = df_test['id']

In [136]:
df_train_id.to_csv('train_id.csv',index=False)
df_test_id.to_csv('test_id.csv',index=False)

In [137]:
df_train_id.shape, df_test_id.shape

((5464,), (1367,))

# Add sentimental analysis

In [141]:
# you need the sentimental file for this

In [140]:
# df_features_train = pd.read_csv("features_train.csv")
# columns_to_remove = ["price"]

# df_features_train = df_features_train.drop(columns=[col for col in columns_to_remove if col in df_features_train.columns])

# df_train= pd.merge(df_train, df_features_train, on='id', how='left')
# df_train.shape

In [ ]:
# df_features_test = pd.read_csv("features_test.csv")
# columns_to_remove = ["price"]

# df_features_test = df_features_test.drop(columns=[col for col in columns_to_remove if col in df_features_test.columns])

# df_test= pd.merge(df_test, df_features_test, on='id', how='left')
# df_test.shape

### Save Data as csv

In [142]:
# Count and print the number of columns by type
bool_cols = df_train.select_dtypes(include='bool').columns.tolist()
cat_cols = df_train.select_dtypes(include=['category']).columns.tolist()
str_cols = df_train.select_dtypes(include=['object', 'string']).columns.tolist()
num_cols = df_train.select_dtypes(include=['int', 'float']).columns.tolist()
other_cols = [col for col in df_train.columns if col not in bool_cols + cat_cols + str_cols + num_cols]

print(f"🟩 Boolean columns: {len(bool_cols)}")
print(f"🟦 Categorical columns: {len(cat_cols)}")
print(f"🟧 Object/String columns: {len(str_cols)}")
print(f"🟨 Numeric columns: {len(num_cols)}")
print(f"🟥 Other columns: {len(other_cols)}")
# Final check
str_obj_cols = df_train.select_dtypes(include=['object', 'string']).columns.tolist()
print(f"🟦 Remaining string/object columns in df: {str_obj_cols}")

🟩 Boolean columns: 184
🟦 Categorical columns: 0
🟧 Object/String columns: 4
🟨 Numeric columns: 36
🟥 Other columns: 0
🟦 Remaining string/object columns in df: ['neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'property_type', 'room_type']


In [ ]:
# df_train.to_csv("train_final.csv", index=False)
# df_test.to_csv("test_final.csv", index=False)

In [143]:
df_train.shape, df_test.shape

((5464, 224), (1367, 224))